In [32]:
import cohere, os
co = cohere.Client(os.getenv("COHERE_API_KEY"))

def generate_query_from_promt(prompt):
    response = co.chat(
        message=prompt,
        search_queries_only=True
    )
    return response.search_queries[0]["text"]

In [33]:
# Example of how to generate a query from a prompt
generate_query_from_promt("Where do the tallest penguins live?")

'tallest penguins'

In [34]:
import weaviate, os

client = weaviate.connect_to_wcs(
    cluster_url=os.getenv("WORKSHOP_DEMO_URL"),
    auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WORKSHOP_DEMO_KEY_READ")),

    headers={
        "X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY"),
        "X-Cohere-Api-Key": os.getenv("COHERE_API_KEY"),

        "X-AWS-Access-Key": os.getenv("AWS_ACCESS_KEY"),
        "X-AWS-Secret-Key": os.getenv("AWS_SECRET_KEY"),
    },

    additional_config=weaviate.config.AdditionalConfig(timeout=(500, 1500))
)

client.is_ready()

True

In [35]:
from weaviate.classes.query import Filter

def two_step_rag(user_prompt):
    # Step 1
    prompt = user_prompt + " Please only use the provided content with this prompt. Don't make things up."
    
    generated_query = generate_query_from_promt(prompt)
    print("=== Generated Query ===")
    print(f"Generated query: {generated_query}")

    # Step 2
    wiki = client.collections.get("Wikipedia")

    response = wiki.generate.near_text(
        query=generated_query,
        auto_limit=1,
        grouped_task=prompt,
        grouped_properties=["text", "title"]
    )

    # Print results
    print("=== Generated Response ===")
    print(response.generated)

    print("=== Source ===")
    for item in response.objects:
        print(item.properties)

In [31]:
# two_step_rag("What wild animals do we know about?")
two_step_rag("How do airplanes fly?")

=== Generated Query ===
Generated query: How do planes fly
=== Generated Response ===
Airplanes fly using a combination of complex flight control systems and the physical design of their wings. The main controls of an airplane allow the pilot to direct the aircraft in the air by controlling the attitude (roll, pitch and yaw) and engine thrust. This is achieved through the use of various components such as ailerons, rudders, flaps, and elevators, which are all part of the airplane's control system.

The wings of an airplane are crucial for its ability to fly. They have flexible surfaces which are stretched across a frame and made rigid by the lift forces exerted by the airflow over them. This design allows the wings to provide lift, a force that counteracts the weight of the airplane and keeps it in the air. Larger aircraft have rigid wing surfaces which provide additional strength.

The design of the wings can vary. For example, some wings have a double-taper planform, a relatively low

In [8]:
two_step_rag("What are the pros and cons of automation using computer?")

=== Generated Query ===
Generated query: pros and cons of automation using computer ?
=== Generated Response ===
Pros of Automation Using Computer:

1. Increased Production Output: As per the data from 2019, the production output of first world nations' manufacturing sectors was doubled when compared to 1984 output. This indicates that automation has significantly increased the efficiency and productivity of industries.

2. Reduced Operating Costs: Automation has led to significantly reduced operating costs. This is because machines can work continuously without the need for breaks, and they do not require benefits or salaries.

3. Less Requirement for Low-Skilled Workers: Automation has the potential to replace jobs that require less than a bachelor's degree. This could lead to a more educated and skilled workforce, as individuals would need to upskill to remain competitive.

Cons of Automation Using Computer:

1. Technological Unemployment: The rate at which jobs are disappearing due

In [ ]:
two_step_rag("How do CPUs work?")